In [677]:
from serde import serde
from serde.json import to_json, from_json
from dataclasses import dataclass
from time import sleep
from scrapping import Voiture
import sklearn
import numpy as np


# Récuperation du json et conversion en quelque chose d'utilisable

In [678]:
with open("donnees.json", "r") as fichier:
    contenu_fichier = fichier.read()

In [679]:
reconstitution = from_json(list[Voiture],contenu_fichier)



In [680]:
reconstitution[0]

Voiture(modele='308', carburant='Diesel', prix=20290, kilometrage=75967, garantie_kilometrage='non garanti', boite_de_vitesse='Automatique', transmission=2, couleur='Blanc', garantie='SPOTICAR PREMIUM', date_mise_circulation=2019, puissance=130, silhouette='Berline', nb_places=5, utilisation_prec='Ex-Loueur', puissance_fiscale=6, critair=2, ptac=1890, nb_portes=5)

# Conversion en base de données panda et nettoyage

In [681]:
import pandas as pd

In [682]:
df = pd.DataFrame(reconstitution)


In [683]:
df.describe()

,prix,kilometrage,date_mise_circulation
count,600.000000,600.000000,600.000000
mean,26750.178333,28932.145000,2020.473333
std,8835.590982,29263.082939,1.738100
min,7490.000000,3.000000,2011.000000
25%,19721.750000,7000.000000,2019.000000
50%,26240.000000,19521.000000,2021.000000
75%,33524.750000,41282.500000,2022.000000
max,59980.000000,176600.000000,2022.000000


In [684]:
df.head()


,modele,carburant,prix,kilometrage,garantie_kilometrage,boite_de_vitesse,transmission,couleur,garantie,date_mise_circulation,puissance,silhouette,nb_places,utilisation_prec,puissance_fiscale,critair,ptac,nb_portes
0,308,Diesel,20290,75967,non garanti,Automatique,2,Blanc,SPOTICAR PREMIUM,2019,130,Berline,5,Ex-Loueur,6,2,1890,5
1,3008,Diesel,31900,29066,non garanti,Manuelle,NA,Gris,SPOTICAR PREMIUM,2021,130,SUV-4x4,5,NA,7,2,NA,5
2,3008,Essence,33920,31997,non garanti,Automatique,NA,Gris,SPOTICAR PREMIUM,2021,130,SUV-4x4,5,NA,7,1,NA,5
3,2008,Essence,21980,40952,non garanti,Manuelle,NA,Gris,SPOTICAR PREMIUM,2021,100,SUV-4x4,5,NA,5,1,NA,5
4,2008,Essence,22990,7901,non garanti,Manuelle,NA,Gris,SPOTICAR PREMIUM,2021,100,SUV-4x4,5,Ex-Particulier,5,1,NA,5


In [685]:
for i in df.columns:
    col_selec = f"df.{i}.unique()"
    
    print(f"{i} : {eval(col_selec)} ")
# retourne toutes les modalités pour chaque variable

modele : ['308' '3008' '2008' '208' '108' '308 SW' '5008' '508' 'RIFTER' 'BOXER'
 '508 SW' 'TRAVELLER' 'PARTNER' 'EXPERT' '4008'] 
carburant : ['Diesel' 'Essence' 'Electrique' 'Hybride rechargeable'] 
prix : [20290 31900 33920 21980 22990 21990 34390 17890 32990 20490 18990 11990
 26290 33490 17490 32490 25990 26590 22490 17690 34490 36990 38790 32590
 28990 25290 16990 22190 23980 33790 38490 22390 26990 29990 38980 31690
 24590 35990 38290 28790 26190 23790 26299 21490 31290 35690 22290 46790
 15490 28980 33091 30790 40490 28490 36390 17990 39790 23220 59980 38990
 24790 23290 32480 20480 34990 27790 23990 15480 23390 52890 29290 27590
 27990 26490 15290 27690 39990 42580 36490 32390 28690 32300 24990 19290
 42590 46290 31580 52190 37990 28590 39390 39690 38280 24490 23150 26390
 37180 21590 35299 38499 37099 24199 42199 14790 14799 37490 39631 47490
 37599 14590 33990 28390 16490 18480 17980 22980 39290 22180 27980 12480
 19280 33480 12280 22280 33980 31780 40980 24280 15380 18580 3

## Remarques sur les variables :

- modele : enlever les espaces, ou supprimer les variables peu occurentes (<=1)
- carburant : même remarque
- même remarque pour toutes les variables textuelles
- utilisation précédente : regrouper par pro/loueur/part/ ne sait pas ?

In [686]:
df.convert_dtypes().dtypes #censé convertir automatiquement vers le type le plus adapté
# mais pas convaincu

modele                   string
carburant                string
prix                      Int64
kilometrage               Int64
garantie_kilometrage     string
boite_de_vitesse         string
transmission             object
couleur                  string
garantie                 string
date_mise_circulation     Int64
puissance                object
silhouette               string
nb_places                object
utilisation_prec         string
puissance_fiscale        object
critair                  object
ptac                     object
nb_portes                object
dtype: object

In [687]:
df.dtypes # types finaux

modele                   object
carburant                object
prix                      int64
kilometrage               int64
garantie_kilometrage     object
boite_de_vitesse         object
transmission             object
couleur                  object
garantie                 object
date_mise_circulation     int64
puissance                object
silhouette               object
nb_places                object
utilisation_prec         object
puissance_fiscale        object
critair                  object
ptac                     object
nb_portes                object
dtype: object

## Création de dummys


In [688]:
df.head()

,modele,carburant,prix,kilometrage,garantie_kilometrage,boite_de_vitesse,transmission,couleur,garantie,date_mise_circulation,puissance,silhouette,nb_places,utilisation_prec,puissance_fiscale,critair,ptac,nb_portes
0,308,Diesel,20290,75967,non garanti,Automatique,2,Blanc,SPOTICAR PREMIUM,2019,130,Berline,5,Ex-Loueur,6,2,1890,5
1,3008,Diesel,31900,29066,non garanti,Manuelle,NA,Gris,SPOTICAR PREMIUM,2021,130,SUV-4x4,5,NA,7,2,NA,5
2,3008,Essence,33920,31997,non garanti,Automatique,NA,Gris,SPOTICAR PREMIUM,2021,130,SUV-4x4,5,NA,7,1,NA,5
3,2008,Essence,21980,40952,non garanti,Manuelle,NA,Gris,SPOTICAR PREMIUM,2021,100,SUV-4x4,5,NA,5,1,NA,5
4,2008,Essence,22990,7901,non garanti,Manuelle,NA,Gris,SPOTICAR PREMIUM,2021,100,SUV-4x4,5,Ex-Particulier,5,1,NA,5


In [689]:
df.dtypes


modele                   object
carburant                object
prix                      int64
kilometrage               int64
garantie_kilometrage     object
boite_de_vitesse         object
transmission             object
couleur                  object
garantie                 object
date_mise_circulation     int64
puissance                object
silhouette               object
nb_places                object
utilisation_prec         object
puissance_fiscale        object
critair                  object
ptac                     object
nb_portes                object
dtype: object

In [690]:
df.ptac

0      1890
1        NA
2        NA
3        NA
4        NA
       ... 
595    1940
596    1770
597    2000
598      NA
599    2790
Name: ptac, Length: 600, dtype: object

In [691]:
df_mod = pd.get_dummies(df["modele"])
df_garkil = pd.get_dummies(df["garantie_kilometrage"])
df_carbu = pd.get_dummies(df["carburant"])
df_bdv = pd.get_dummies(df["boite_de_vitesse"])
df_couleur = pd.get_dummies(df["couleur"])
df_sil = pd.get_dummies(df["silhouette"])
df_trans = pd.get_dummies(df["transmission"])
df_gar = pd.get_dummies(df["garantie"])
df_utilprec = pd.get_dummies(df["utilisation_prec"])

df = pd.concat([df, df_mod, df_garkil, df_carbu, df_bdv, df_couleur, df_sil,df_trans, df_gar, df_utilprec], axis = 1)

del df['modele']
del df['garantie_kilometrage']
del df['carburant']
del df['boite_de_vitesse']
del df['couleur']
del df['silhouette']
del df['transmission']
del df['garantie']
del df['utilisation_prec']
del df['NA']

In [692]:
df.convert_dtypes().dtypes

prix                      Int64
kilometrage               Int64
date_mise_circulation     Int64
puissance                object
nb_places                object
puissance_fiscale        object
critair                  object
ptac                     object
nb_portes                object
108                       UInt8
2008                      UInt8
208                       UInt8
3008                      UInt8
308                       UInt8
308 SW                    UInt8
4008                      UInt8
5008                      UInt8
508                       UInt8
508 SW                    UInt8
BOXER                     UInt8
EXPERT                    UInt8
PARTNER                   UInt8
RIFTER                    UInt8
TRAVELLER                 UInt8
garanti                   UInt8
non garanti               UInt8
Diesel                    UInt8
Electrique                UInt8
Essence                   UInt8
Hybride rechargeable      UInt8
Automatique               UInt8
Manuelle

In [693]:
df = df.replace('NA', np.NaN)
df_EN = df.dropna()

## Passage sous numpy + ML

In [694]:
y = df_EN['prix'].to_numpy()
X = df_EN.loc[:, df_EN.columns != 'prix'].to_numpy() # tout sauf le prix

In [695]:
type(y), y.shape

(numpy.ndarray, (515,))

In [696]:
type(X), X.shape

(numpy.ndarray, (515, 59))

In [697]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import OneHotEncoder

In [698]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y)

In [699]:
en = ElasticNet()
en_gs = GridSearchCV(
    en,
    {
        "alpha": [2 ** p  for p in range(-6, 6)],
        "l1_ratio": (0.01, 0.25, 0.5, 0.75, 1),
    }
)
en_gs.fit(X_tr, y_tr) #problème vient du fit
en_df = pd.DataFrame(en_gs.cv_results_)
en_df.head()
        

c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.466e+08, tolerance: 2.590e+06
  model = cd_fast.enet_coordinate_descent(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.599e+08, tolerance: 2.416e+06
  model = cd_fast.enet_coordinate_descent(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.024135,0.003117,0.000399,0.000489,0.015625,0.01,"{'alpha': 0.015625, 'l1_ratio': 0.01}",0.934492,0.904828,0.925663,0.920569,0.941237,0.925358,0.012491,4
1,0.021223,0.008695,0.000560,0.000463,0.015625,0.25,"{'alpha': 0.015625, 'l1_ratio': 0.25}",0.933906,0.905451,0.922978,0.919666,0.942527,0.924905,0.012658,8
2,0.019178,0.004644,0.000399,0.000489,0.015625,0.5,"{'alpha': 0.015625, 'l1_ratio': 0.5}",0.932571,0.906218,0.918533,0.917495,0.943913,0.923746,0.013100,10
3,0.029875,0.007178,0.000399,0.000489,0.015625,0.75,"{'alpha': 0.015625, 'l1_ratio': 0.75}",0.929664,0.907270,0.909422,0.911525,0.945393,0.920655,0.014709,16
4,0.017285,0.012116,0.000200,0.000400,0.015625,1,"{'alpha': 0.015625, 'l1_ratio': 1}",0.919488,0.909984,0.833191,0.722160,0.947634,0.866491,0.081517,43


In [700]:
en_gs.best_params_, en_gs.best_score_

({'alpha': 0.03125, 'l1_ratio': 0.25}, 0.9254340652086996)

In [701]:
from sklearn.neighbors import KNeighborsRegressor

KNN

In [702]:
knr = KNeighborsRegressor()
knr_gs = GridSearchCV(
    knr,
    {
        "n_neighbors": range(5, 15),
        "weights": ("uniform", "distance"),
    }
)

In [703]:
knr_gs.fit(X_tr, y_tr)
knr_df = pd.DataFrame(knr_gs.cv_results_)
knr_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000603,0.000492,0.009182,0.001170,5,uniform,"{'n_neighbors': 5, 'weights': 'uniform'}",0.472891,0.454638,0.454651,0.620872,0.329426,0.466495,0.092674,11
1,0.000798,0.000399,0.010166,0.002020,5,distance,"{'n_neighbors': 5, 'weights': 'distance'}",0.530809,0.541041,0.567135,0.708267,0.435529,0.556556,0.087954,1
2,0.000803,0.000402,0.006777,0.000762,6,uniform,"{'n_neighbors': 6, 'weights': 'uniform'}",0.481322,0.433027,0.459184,0.596868,0.316208,0.457322,0.090062,12
3,0.000804,0.000402,0.008372,0.000481,6,distance,"{'n_neighbors': 6, 'weights': 'distance'}",0.554002,0.520710,0.568064,0.700154,0.424451,0.553476,0.088824,2
4,0.000199,0.000398,0.006384,0.001028,7,uniform,"{'n_neighbors': 7, 'weights': 'uniform'}",0.479794,0.423591,0.421023,0.584859,0.286973,0.439248,0.096552,13


In [704]:
knr_gs.best_params_, knr_gs.best_score_

({'n_neighbors': 5, 'weights': 'distance'}, 0.556556133091278)

In [705]:
from sklearn.ensemble import RandomForestRegressor

In [706]:
rfr = RandomForestRegressor()
rfr_gs = GridSearchCV(
    rfr,
    {   
        "n_estimators": (8 , 16, 32, 64, 128, 256),
    }
)

In [707]:
rfr_gs.fit(X_tr, y_tr)
rfr_df = pd.DataFrame(rfr_gs.cv_results_)
rfr_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.028719,0.001595,0.001596,0.000798,8,{'n_estimators': 8},0.893452,0.817990,0.907110,0.893903,0.935982,0.889687,0.039036,6
1,0.059413,0.004518,0.001993,0.001091,16,{'n_estimators': 16},0.916545,0.806639,0.925112,0.901289,0.939762,0.897869,0.047288,5
2,0.097559,0.013570,0.003378,0.001008,32,{'n_estimators': 32},0.929495,0.830699,0.936286,0.906946,0.936928,0.908071,0.040188,4
3,0.174268,0.035891,0.004788,0.000746,64,{'n_estimators': 64},0.928200,0.841467,0.943179,0.907672,0.945930,0.913290,0.038396,1
4,0.399125,0.059336,0.009775,0.000746,128,{'n_estimators': 128},0.925514,0.835618,0.945782,0.902729,0.945008,0.910930,0.040811,2


In [708]:
rfr_gs.best_params_, rfr_gs.best_score_

({'n_estimators': 64}, 0.9132896722202967)

In [709]:
from sklearn.svm import SVR

In [710]:
svr = SVR()
svr_gs = GridSearchCV(
    svr,
    {
        "C": (0.1, 1.0, 10),
        "epsilon": (0.1, 1.0, 10),
    }
)
svr_gs.fit(X_tr, y_tr)

svr_df = pd.DataFrame(svr_gs.cv_results_)
svr_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_epsilon,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009961,0.002660,0.004589,0.000799,0.1,0.1,"{'C': 0.1, 'epsilon': 0.1}",-0.005452,-0.038538,-0.006856,-0.001036,-0.005375,-0.011451,0.013684,7
1,0.007779,0.001323,0.003391,0.000489,0.1,1.0,"{'C': 0.1, 'epsilon': 1.0}",-0.005452,-0.038547,-0.006865,-0.001036,-0.005380,-0.011456,0.013686,8
2,0.006183,0.000399,0.003192,0.000399,0.1,10,"{'C': 0.1, 'epsilon': 10}",-0.005356,-0.038903,-0.006750,-0.001036,-0.005546,-0.011518,0.013829,9
3,0.006983,0.000889,0.002990,0.000627,1.0,0.1,"{'C': 1.0, 'epsilon': 0.1}",-0.001654,-0.036696,-0.004369,0.002338,-0.002763,-0.008629,0.014207,5
4,0.006396,0.000493,0.003186,0.000403,1.0,1.0,"{'C': 1.0, 'epsilon': 1.0}",-0.001654,-0.036696,-0.004369,0.002338,-0.002763,-0.008629,0.014207,5


In [711]:
svr_gs.best_params_, svr_gs.best_score_

({'C': 10, 'epsilon': 0.1}, 0.020261334629391968)

In [712]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

In [713]:
pl = Pipeline(
    [
        ("mise_echelle", MinMaxScaler()),
        ("support_vecteurs", SVR()),
    ]
)

In [714]:
pl_gs = GridSearchCV(
    pl,
    {
        "support_vecteurs__C": (0.1, 1.0, 10),
        "support_vecteurs__epsilon": (0.1, 1.0, 10),
    }
)

In [715]:
pl_gs.fit(X_tr, y_tr)

pl_df = pd.DataFrame(pl_gs.cv_results_)
pl_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_support_vecteurs__C,param_support_vecteurs__epsilon,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009379,0.000796,0.004591,0.000796,0.1,0.1,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",-0.005578,-0.038796,-0.006828,-0.001141,-0.005367,-0.011542,0.013761,7
1,0.008376,0.001017,0.004590,0.001017,0.1,1.0,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",-0.005572,-0.038796,-0.006846,-0.001141,-0.005367,-0.011544,0.013760,8
2,0.010973,0.001667,0.003784,0.000736,0.1,10,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",-0.005420,-0.038810,-0.006861,-0.001141,-0.005532,-0.011553,0.013764,9
3,0.010571,0.001738,0.004790,0.000747,1.0,0.1,"{'support_vecteurs__C': 1.0, 'support_vecteurs...",-0.002905,-0.036377,-0.004052,0.001289,-0.002661,-0.008941,0.013836,4
4,0.009973,0.002522,0.004589,0.001620,1.0,1.0,"{'support_vecteurs__C': 1.0, 'support_vecteurs...",-0.002905,-0.036377,-0.004060,0.001289,-0.002661,-0.008943,0.013836,5


In [716]:
pl_gs.best_params_, pl_gs.best_score_

({'support_vecteurs__C': 10, 'support_vecteurs__epsilon': 10},
 0.016877606845633088)

In [717]:
from sklearn.neural_network import MLPRegressor
import numpy as np

In [718]:
pln = Pipeline(
    [
        ("mise_echelle", MinMaxScaler()),
        ("neurones", MLPRegressor()),
    ]
)

In [719]:
pln_gs = GridSearchCV(
    pln,
    {
        "neurones__alpha": 10.0 ** -np.arange(1, 7),
        'neurones__hidden_layer_sizes': ((25,), (50, ), (100,), (20, 20)),
    }
)

In [720]:
pln_gs.fit(X_tr, y_tr)

pln_df = pd.DataFrame(pln_gs.cv_results_)
pln_df.head()

c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the opti

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_neurones__alpha,param_neurones__hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.336750,0.031155,0.001003,0.000634,0.1,"(25,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-8.962990,-8.087020,-10.468480,-7.565196,-10.730245,-9.162786,1.257875,19
1,0.642041,0.087698,0.001004,0.001094,0.1,"(50,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-8.897520,-8.028584,-10.378046,-7.530007,-10.660117,-9.098855,1.242672,13
2,0.663623,0.010594,0.000993,0.000009,0.1,"(100,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-8.806285,-7.932751,-10.286417,-7.444113,-10.556126,-9.005138,1.238838,11
3,0.152185,0.007073,0.000609,0.000497,0.1,"(20, 20)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-8.294474,-7.520076,-9.912753,-7.260062,-9.930627,-8.583598,1.144339,6
4,0.252099,0.002700,0.000605,0.000495,0.01,"(25,)","{'neurones__alpha': 0.01, 'neurones__hidden_la...",-8.967737,-8.095142,-10.475290,-7.564567,-10.721769,-9.164901,1.255809,20


In [721]:
pln_gs.best_params_

pln_gs.best_score_

-8.484806088946526

In [722]:
pln_gs.best_params_

{'neurones__alpha': 1e-05, 'neurones__hidden_layer_sizes': (20, 20)}

In [723]:
essai = en_gs.best_estimator_

In [724]:
prix_prédits = essai.predict(X)

In [725]:
prix_reels = y

In [726]:
prix_reels,prix_prédits

(array([20290, 34390, 17890, 32990, 20490, 18990, 26290, 33490, 17490,
        32490, 26590, 26290, 22490, 17690, 34490, 36990, 38790, 18990,
        32590, 28990, 25290, 16990, 22190, 23980, 33790, 38490, 22390,
        22490, 26990, 29990, 38980, 25990, 31690, 24590, 35990, 38290,
        28790, 32490, 26190, 34490, 23790, 26299, 38290, 35990, 21490,
        25990, 31290, 35690, 22290, 46790, 15490, 28980, 21990, 21990,
        33091, 30790, 40490, 28490, 36390, 17990, 39790, 17990, 38490,
        23220, 22190, 59980, 38990, 24790, 23290, 32480, 20480, 34990,
        24590, 27790, 38980, 28490, 23990, 26590, 22190, 15480, 23390,
        34490, 52890, 29290, 27590, 27990, 26490, 34490, 32490, 15290,
        35690, 27690, 35690, 39990, 42580, 36490, 26190, 32390, 35990,
        28690, 32300, 28990, 24990, 26590, 19290, 29990, 42590, 46290,
        31580, 52190, 42590, 37990, 28590, 19290, 39390, 39990, 39690,
        26990, 40490, 38280, 24490, 23150, 37180, 21590, 35299, 24490,
      

In [727]:
len(prix_prédits)

515

In [728]:
len(prix_reels)

515

In [729]:
prix_prédits

array([21030.81380811, 34683.63767931, 18531.17009031, 33497.28694387,
       19040.36464868, 20430.3745238 , 27751.03982465, 33672.02546657,
       18810.86347411, 32304.44143869, 26530.33587074, 29129.6562666 ,
       21919.56616851, 18575.45226579, 34547.38390762, 39662.65788713,
       35629.93803586, 19973.58871083, 32347.12200657, 30153.61499349,
       26896.97294569, 17384.84077182, 22980.51295976, 25130.93667649,
       31729.05913926, 36111.94519909, 22772.62647661, 22963.18481035,
       28422.17169967, 28150.87957626, 37217.80592599, 28689.08590803,
       30058.63262413, 25649.75827743, 33705.26157238, 37358.90181046,
       29861.91924681, 31307.96496106, 27666.7711695 , 33555.06667932,
       24602.90798135, 26632.38331423, 36836.36437342, 36780.58261545,
       20672.70100929, 28844.40574513, 32935.17255314, 33361.62109107,
       22605.73669781, 35253.50093206, 16240.03573484, 30051.53868192,
       19915.60972459, 23392.17764291, 37945.73185356, 31171.35797123,
      

In [730]:
prix_reels

array([20290, 34390, 17890, 32990, 20490, 18990, 26290, 33490, 17490,
       32490, 26590, 26290, 22490, 17690, 34490, 36990, 38790, 18990,
       32590, 28990, 25290, 16990, 22190, 23980, 33790, 38490, 22390,
       22490, 26990, 29990, 38980, 25990, 31690, 24590, 35990, 38290,
       28790, 32490, 26190, 34490, 23790, 26299, 38290, 35990, 21490,
       25990, 31290, 35690, 22290, 46790, 15490, 28980, 21990, 21990,
       33091, 30790, 40490, 28490, 36390, 17990, 39790, 17990, 38490,
       23220, 22190, 59980, 38990, 24790, 23290, 32480, 20480, 34990,
       24590, 27790, 38980, 28490, 23990, 26590, 22190, 15480, 23390,
       34490, 52890, 29290, 27590, 27990, 26490, 34490, 32490, 15290,
       35690, 27690, 35690, 39990, 42580, 36490, 26190, 32390, 35990,
       28690, 32300, 28990, 24990, 26590, 19290, 29990, 42590, 46290,
       31580, 52190, 42590, 37990, 28590, 19290, 39390, 39990, 39690,
       26990, 40490, 38280, 24490, 23150, 37180, 21590, 35299, 24490,
       38499, 37990,

Objectif : maximiser le surplus de l'acheteur, i.e, si l'acheteur à un budget R, et n contraintes, quel véhicule maximiserait son surplus


In [734]:
data_finale = df_EN

In [741]:
data_finale['prix_predits'] = prix_prédits
data_finale['surplus_conso'] = data_finale['prix'] - data_finale['prix_predits']

C:\Users\mathi\AppData\Local\Temp\ipykernel_10928\2711882043.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_finale['prix_predits'] = prix_prédits
C:\Users\mathi\AppData\Local\Temp\ipykernel_10928\2711882043.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_finale['surplus_conso'] = data_finale['prix'] - data_finale['prix_predits']


In [759]:
data_finale

,prix,kilometrage,date_mise_circulation,puissance,nb_places,puissance_fiscale,critair,ptac,nb_portes,108,...,SPOTICAR ESSENTIAL,SPOTICAR PREMIUM,Ex-Auto-école,Ex-Import,Ex-Loueur,Ex-Particulier,Ex-Société,Véhicule de direction,prix_predits,surplus_conso
0,20290,75967,2019,130.0,5.0,6.0,2.0,1890.0,5.0,0,...,0,1,0,0,1,0,0,0,21030.813808,-740.813808
6,34390,24889,2022,131.0,5.0,7.0,2.0,2000.0,5.0,0,...,0,1,0,0,1,0,0,0,34683.637679,-293.637679
7,17890,19422,2020,102.0,5.0,5.0,1.0,1595.0,5.0,0,...,0,1,0,0,1,0,0,0,18531.170090,-641.170090
8,32990,38139,2022,131.0,5.0,7.0,2.0,2000.0,5.0,0,...,0,1,0,0,1,0,0,0,33497.286944,-507.286944
9,20490,50816,2019,130.0,5.0,6.0,1.0,1750.0,5.0,0,...,0,1,0,1,0,0,0,0,19040.364649,1449.635351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,17790,14000,2021,75.0,5.0,4.0,1.0,1510.0,5.0,0,...,0,1,0,0,1,0,0,0,17795.111172,-5.111172
594,32990,9007,2021,136.0,5.0,4.0,0.0,2030.0,5.0,0,...,0,1,0,0,0,0,0,0,32858.704315,131.295685
595,33990,3254,2022,131.0,5.0,7.0,2.0,1940.0,5.0,0,...,0,1,0,0,0,0,0,0,33757.383455,232.616545
596,31490,2535,2022,131.0,5.0,7.0,2.0,1770.0,5.0,0,...,0,1,0,1,0,0,0,0,32279.100483,-789.100483


In [737]:
newdf =data_finale.loc[
    (data_finale["208"] == 1) & 
    (data_finale["Essence"]) & 
    (data_finale['Gris']) &
    (data_finale["Citadine"]) &
    (data_finale['2') &
    (data_finale.date_mise_circulation >= 2017) 
    ]

AttributeError: 'DataFrame' object has no attribute 'modele'

In [ ]:
newdf.head()

,modele,carburant,prix,kilometrage,garantie_kilometrage,boite_de_vitesse,transmission,couleur,garantie,date_mise_circulation,puissance,silhouette,nb_places,utilisation_prec,puissance_fiscale,critair,ptac,nb_portes,prix_predits,surplus_conso


In [ ]:
meilleur_choix = newdf.loc[newdf['surplus_conso'] == newdf['surplus_conso'].max()]

In [ ]:
tri_meilleures = newdf.sort_values(by = 'surplus_conso', ascending = False).head()

In [ ]:
tri_meilleures

,modele,carburant,prix,kilometrage,garantie_kilometrage,boite_de_vitesse,transmission,couleur,garantie,date_mise_circulation,puissance,silhouette,nb_places,utilisation_prec,puissance_fiscale,critair,ptac,nb_portes,prix_predits,surplus_conso


In [ ]:
tri_meilleures.index

Int64Index([], dtype='int64')